In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import cv2
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader

from glob import glob

from src.models import FPathPredictor
from src.dataloaders import FPathDataset
from src.utils import inference
from src.preprocesses import VTFPreprocessor

In [ ]:
vtf_paths = glob("/home/joono/fpath_infodraw/dataset/simple_data/test/vtfs/*")
vtf_paths.sort()
vtfs      = [VTFPreprocessor.get_vtf(vtf_path) for vtf_path in vtf_paths]

In [ ]:
weight_path = "/home/joono/fpath_infodraw/checkpoints/best_model_epoch9.pt"

model = FPathPredictor()
model.load_state_dict(torch.load(weight_path))

model = model.to("cuda")
model = model.eval()

In [ ]:
vtf, vtf_path = vtfs[1], vtf_paths[1]

vtf = torch.tensor(vtf).to('cuda')
pred = inference(model, vtf.unsqueeze(0))

result = pred.squeeze().detach().cpu().numpy()
result = result.transpose((1, 0))
# plt.imshow(result, cmap="gray")

cv2.imwrite(f"{os.path.basename(vtf_path).split('.')[0]}.png", result * 255)

In [5]:
from src.dataloaders import get_data_loaders, get_FPathUNetDataset, FPathUNetDataset

In [6]:
dset = FPathUNetDataset(config_path="dataset/test.yaml")

In [10]:
vtf, img, target = dset[0]

In [11]:
vtf.shape, img.shape, target.shape

((21, 1024, 1024), torch.Size([3, 1024, 1024]), torch.Size([1, 1024, 1024]))

In [16]:
import numpy as np
np.sum(vtf == 1.0), np.sum(vtf < 1.0)

(20558826, 1461270)

In [17]:
1024 * 1024 * 21

22020096